In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium

# Load the GeoJSON file with country boundaries
world_geojson = gpd.read_file('world_country_borders.geojson')  # Replace with your file path

# Load the population data (CSV or Excel file)
CountryData = pd.read_csv('CountryData.csv')  # Replace with your data file path

# Apply logarithmic transformation to the population data
CountryData['Log_Pop'] = np.log1p(CountryData['Pop 2022'])

#New Data
CountryData['pop/km'] = CountryData['Pop 2022']/CountryData['Area SQKM']
CountryData['log_pop/km'] = np.log1p(CountryData['pop/km'])

# Merge the GeoJSON and population data based on a common identifier (e.g., ISO country code)
merged_data = world_geojson.merge(CountryData, left_on='iso_a3', right_on='Country_Code', how='left')

# Filter down to Africa
merged_data = merged_data[merged_data['Cont']=='AF']

# Save the merged data as a new GeoJSON file
merged_data.to_file('world_population.geojson', driver='GeoJSON')


# Load GeoJSON data
geojson_file = 'world_population.geojson'  # Replace with your GeoJSON file path

# Create a Folium Map centered at a specific location
m = folium.Map(location=[0, 20], zoom_start=4)

# Create a choropleth map layer
folium.Choropleth(
    geo_data=geojson_file,
    name='choropleth',
    data=CountryData,
    columns=['Country_Code', 'log_pop/km'],  # Replace with your data column names
    key_on='feature.properties.iso_a3',  # Replace with the GeoJSON property path
    fill_color='YlGn',  # Color scale (you can choose different colors)
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Log Pop / km',
).add_to(m)

folium.features.GeoJson('world_population.geojson', name="Population", popup=folium.features.GeoJsonPopup(fields=['country','Log_Pop','Pop 2022','Area SQKM','pop/km'])).add_to(m)

# Add a Layer Control to toggle between layers
folium.LayerControl().add_to(m)

# Display the map
m.save('population_choropleth_map.html')  # Save the map to an HTML file
m 

ModuleNotFoundError: No module named 'geopandas'